# Naive Bayes with Mushrooms

In [2]:
import pandas as pd
#讀取CSV檔案 轉成dataFrame
data = pd.read_csv('mushrooms.csv')  

,type,cap_shape,cap_surface,cap_color,odor,stalk_shape,stalk_color_above_ring,stalk_color_below_ring,ring_number,ring_type,population,habitat
0,p,x,s,n,p,e,w,w,o,p,s,u
1,e,x,s,y,a,e,w,w,o,p,n,g
2,e,b,s,w,l,e,w,w,o,p,n,m
3,p,x,y,w,p,e,w,w,o,p,s,u
4,e,x,s,g,n,t,w,w,o,e,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,n,e,o,o,o,p,c,l
8120,e,x,s,n,n,e,o,o,o,p,v,l
8121,e,f,s,n,n,e,o,o,o,p,c,l
8122,p,k,y,n,y,t,w,w,o,e,v,l


## info ( )
主要可以看有幾筆資料、每個欄位的資料型別是什麼(int, float..)、有無空值(null)的存在、佔據多少記憶體
## describe ( )
主要是看資料的平均值、分佈情況、是否有資料傾斜Skew的問題

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 12 columns):
type                      8124 non-null object
cap_shape                 8124 non-null object
cap_surface               8124 non-null object
cap_color                 8124 non-null object
odor                      8124 non-null object
stalk_shape               8124 non-null object
stalk_color_above_ring    8124 non-null object
stalk_color_below_ring    8124 non-null object
ring_number               8124 non-null object
ring_type                 8124 non-null object
population                8124 non-null object
habitat                   8124 non-null object
dtypes: object(12)
memory usage: 761.8+ KB


In [ ]:
data

In [4]:
data.describe()

,type,cap_shape,cap_surface,cap_color,odor,stalk_shape,stalk_color_above_ring,stalk_color_below_ring,ring_number,ring_type,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,9,2,9,9,3,5,6,7
top,e,x,y,n,n,t,w,w,o,p,v,d
freq,4208,3656,3244,2284,3528,4608,4464,4384,7488,3968,4040,3148


## 切分input 和output
把data切成2個dataFrame，分別是input和output

In [14]:
#x:input
x=data.iloc[:,1:12]

#y:output
y=data.loc[:,['type']]

## sklearn: Naive Bayes Classifier

In [32]:
from sklearn import preprocessing

#將屬性轉為數字label
le = preprocessing.LabelEncoder()

columns = []
for c in x:
    encoded = le.fit_transform(data[c])
    columns.append(encoded)

#將play轉為數字label
#play: no: 0 ,yes: 1
Y_type_label=le.fit_transform(y.type)

#將屬性合併
#變成list
feature=list(zip(columns[0], columns[1], columns[2], columns[3], 
                 columns[4], columns[5], columns[6], columns[7], 
                 columns[8], columns[9], columns[10]))

#轉成array
import numpy as np
features=np.asarray(feature)

In [46]:
features

array([[5, 2, 4, ..., 4, 3, 5],
       [5, 2, 9, ..., 4, 2, 1],
       [0, 2, 8, ..., 4, 2, 3],
       ...,
       [2, 2, 4, ..., 4, 1, 2],
       [3, 3, 4, ..., 0, 4, 2],
       [5, 2, 4, ..., 4, 1, 2]])

## 訓練模型：訓練集

In [35]:
#Import Gaussian Naive Bayes 模型 (高斯樸素貝氏)
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

# 訓練集訓練模型
# model.fit(x, y)
model.fit(features, Y_type_label)

GaussianNB(priors=None, var_smoothing=1e-09)

## 測試集測試模型

In [38]:
expected = Y_type_label
predicted = model.predict(features)

from sklearn import metrics
print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       0.87      0.78      0.82      4208
           1       0.79      0.87      0.83      3916

    accuracy                           0.83      8124
   macro avg       0.83      0.83      0.83      8124
weighted avg       0.83      0.83      0.83      8124



In [41]:
print(metrics.confusion_matrix(expected, predicted))

[[3296  912]
 [ 493 3423]]


In [48]:
# 丟一筆資料，是否是能吃的香菇
predicted= model.predict([[0, 2, 9, 0, 0, 7, 7, 1, 4, 3, 3]]) # 抄features[9]的attr
print ("Predicted Value:", predicted)

Predicted Value: [1]
